# TODO: Document
# TODO: Cleanup
* Note, this script may take several days to run
* TODO: Document the related-signal residualization
* TODO: Document that this signal is prone to underfitting

In [1]:
# CHANGE THIS PARAMETER
username = "taapaye"

In [2]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import seaborn as sns
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
from tqdm import tqdm as tqdm


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [3]:
np.random.seed(0)

In [4]:
os.chdir(f"../../data/recommendations/{username}")

In [5]:
filtered_df = pickle.load(open("../../processed_data/user_anime_lists.pkl", "rb"))
user_df = pickle.load(open("user_anime_list.pkl", "rb"))
filtered_df = filtered_df.loc[lambda x: ~x["username"].isin(user_df.username)]
filtered_df = pd.concat([filtered_df, user_df], ignore_index=True)
filtered_df = filtered_df.set_index("username")

In [6]:
related_df = pickle.load(open("related_loocv.pkl", "rb"))

In [7]:
@functools.lru_cache()
def get_item_corrs(nonneg_corrs=False):
    corrs = pickle.load(open("../../processed_data/item_correlations.pkl", "rb"))
    if nonneg_corrs:
        corrs["similarity"] = corrs["corr"]
    else:
        corrs["similarity"] = corrs["corr"].abs()
    corrs = corrs.dropna()
    corrs = corrs.loc[
        lambda x: x.index.get_level_values("anime_id_x")
        != x.index.get_level_values("anime_id_y")
    ]
    corrs = corrs.sort_values(by="similarity")
    return corrs


def get_item_corrs_wrapper(df, username):
    return get_item_corrs()


def get_item_scores(df, corrs, username, neighborhood_size):
    corrs = corrs.groupby("anime_id_x").tail(neighborhood_size)
    score = df.loc[username].merge(
        corrs.reset_index("anime_id_x"), left_on="anime_id", right_on="anime_id_y",
    )
    score = score.drop("anime_id", axis=1).rename({"anime_id_x": "anime_id"}, axis=1)
    return score

In [8]:
def get_delta(score):
    return score.groupby("anime_id").apply(
        lambda x: np.dot(x["score"], x["corr"]) / x["corr"].abs().sum()
    )

In [9]:
def get_squared_error(df, pred_df, username):
    pred_df = pred_df.loc[pred_df.index.intersection(df.loc[username].anime_id)]
    pred_df = pred_df.merge(
        df.loc[username].set_index("anime_id")["score"], on="anime_id"
    )
    errors = pred_df["pred_score"] - pred_df["score"]
    return np.dot(errors, errors)

In [22]:
def compute_accuracy_metrics(
    is_df, oos_df, score_fn, corrs_fn, username, neighborhood_sizes,
):
    corrs = corrs_fn(is_df, username)
    metrics = pd.DataFrame()
    for neighborhood_size in tqdm(
        sorted(neighborhood_sizes), total=len(neighborhood_sizes),
    ):
        score = score_fn(is_df, corrs, username, neighborhood_size)
        pred_df = pd.DataFrame()
        pred_df["related"] = related_df['delta']        
        pred_df["delta"] = get_delta(score)
        pred_df = pred_df.fillna(0)

        # train linear model
        seen_shows = is_df.loc[username].merge(pred_df, on='anime_id')
        model = lm("score ~ delta + related", seen_shows)

        # inference
        pred_df["pred_score"] = model.predict(pred_df)
        return pred_df
        is_pred_df = pred_df.loc[lambda x: x.index.isin(is_df.loc[username].anime_id)]
        oos_pred_df = pred_df.loc[lambda x: x.index.isin(oos_df.anime_id)]

        # compute coverage
        is_coverage = len(is_pred_df.loc[lambda x: ~np.isclose(x['delta'], 0)]) / len(is_df.loc[username])
        oos_coverage = len(oos_pred_df.loc[lambda x: ~np.isclose(x['delta'], 0)]) / len(oos_df)

        # compute rmse
        # TODO delete the 'missing' components as nothing should be missing anymore
        missing_is = is_df.loc[username].loc[
            lambda x: ~x.anime_id.isin(is_pred_df.index)
            & ~x.anime_id.isin(oos_df.anime_id)
        ]
        missing_oos = oos_df.loc[lambda x: ~x.anime_id.isin(oos_pred_df.index)]
        is_se = get_squared_error(is_df, is_pred_df, username)
        oos_se = get_squared_error(oos_df, oos_pred_df, username)
        missing_is_se = np.dot(missing_is["score"], missing_is["score"])
        missing_oos_se = np.dot(missing_oos["score"], missing_oos["score"])
        is_rmse = np.sqrt((is_se + missing_is_se) / len(is_df.loc[username]))
        oos_rmse = np.sqrt((oos_se + missing_oos_se) / len(oos_df))
        metrics = metrics.append(
            {
                "neighborhood_size": neighborhood_size,
                "is_rsquared": model.rsquared_adj,
                "is_rmse": is_rmse,
                "is_coverage": is_coverage,
                "oos_rmse": oos_rmse,
                "oos_coverage": oos_coverage,
            },
            ignore_index=True,
        )
        print({
                "neighborhood_size": neighborhood_size,
                "is_rsquared": model.rsquared_adj,
                "is_rmse": is_rmse,
                "is_coverage": is_coverage,
                "oos_rmse": oos_rmse,
                "oos_coverage": oos_coverage,
            },)
    return metrics

In [23]:
a = compute_accuracy_metrics(
        is_df.copy(),
        oos_df.copy(),
        get_item_scores,
        get_item_corrs_wrapper,
        username,
        [16],
    )

  0%|          | 0/1 [00:37<?, ?it/s]


In [24]:
b = compute_accuracy_metrics(
        is_df.copy(),
        oos_df.copy(),
        get_item_scores,
        get_item_corrs_wrapper,
        username,
        [1448],
    )

  0%|          | 0/1 [01:05<?, ?it/s]


In [27]:
a['delta2'] = b['delta']

In [31]:
        seen_shows = is_df.loc[username].merge(a, on='anime_id')

In [32]:
seen_shows

,anime_id,score,score_var,related,delta,pred_score,delta2
0,1,0.069825,1.812764,0.000000,-1.150561,-0.766137,-0.223729
1,20,-0.775816,2.073848,-1.215833,-0.670547,-0.814251,-0.211051
2,21,-1.705130,2.224881,-1.424083,-0.681599,-0.875114,-0.384288
3,30,0.531055,2.199041,0.398171,-0.353887,-0.212205,-0.170843
4,32,0.398171,2.255429,0.531055,-0.446718,-0.229722,-0.153287
...,...,...,...,...,...,...,...
344,36474,-4.250706,1.425790,-0.610481,0.000000,-0.277132,0.000000
345,36754,-2.131545,1.600437,0.000000,0.000000,-0.116990,-0.516631
346,36838,-0.004645,1.843033,0.005536,-0.063714,-0.151486,-0.468347
347,36949,-2.622063,1.474053,0.044124,-0.785637,-0.548672,-0.600962


In [60]:
print(lm('score~delta + related + delta2', seen_shows).summary())

                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     28.64
Date:                Sat, 29 May 2021   Prob (F-statistic):           1.47e-16
Time:                        02:58:09   Log-Likelihood:                -552.10
No. Observations:                 349   AIC:                             1112.
Df Residuals:                     345   BIC:                             1128.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0671      0.103      0.651      0.5

In [48]:
model = lm('score~delta ', seen_shows)

In [53]:
is_df

,anime_id,score,score_var
username,,,
karthiga,21,0.605479,2.031881
karthiga,59,-0.474160,1.593917
karthiga,74,-0.750239,1.456926
karthiga,120,-0.743815,1.658394
karthiga,178,-0.206618,1.556040
...,...,...,...
taapaye,36474,-4.250706,1.425790
taapaye,36754,-2.131545,1.600437
taapaye,36838,-0.004645,1.843033


In [54]:
model.predict(a).loc[30240]

KeyError: 30240

In [55]:
a

,related,delta,pred_score,delta2
anime_id,,,,
1,0.000000,-1.150561,-0.766137,-0.223729
20,-1.215833,-0.670547,-0.814251,-0.211051
21,-1.424083,-0.681599,-0.875114,-0.384288
30,0.398171,-0.353887,-0.212205,-0.170843
32,0.531055,-0.446718,-0.229722,-0.153287
...,...,...,...,...
36474,-0.610481,0.000000,-0.277132,0.000000
36754,0.000000,0.000000,-0.116990,-0.516631
36838,0.005536,-0.063714,-0.151486,-0.468347


In [21]:
errors_by_neighborhood_size = []
max_size = len(filtered_df.anime_id.unique())
base = np.sqrt(2)
neighborhood_sizes = [
    int(base ** i) for i in range(int(np.log(max_size) / np.log(base)) + 1)
] + [max_size]
neighborhood_sizes = sorted(list(set(neighborhood_sizes)))

oos_df = filtered_df.loc[username].sample(frac=1)
is_df = filtered_df
    
item_metrics = compute_accuracy_metrics(
        is_df.copy(),
        oos_df.copy(),
        get_item_scores,
        get_item_corrs_wrapper,
        username,
        neighborhood_sizes,
    )
item_metrics["signal"] = "item"
errors_by_neighborhood_size.append(item_metrics)    

  4%|▎         | 1/27 [00:34<15:06, 34.88s/it]

{'neighborhood_size': 1, 'is_rsquared': 0.003358980009292245, 'is_rmse': 1.337587450812027, 'is_coverage': 0.6246418338108882, 'oos_rmse': 1.337587450812027, 'oos_coverage': 0.6246418338108882}


  7%|▋         | 2/27 [01:06<13:45, 33.01s/it]

{'neighborhood_size': 2, 'is_rsquared': 0.009961551397987045, 'is_rmse': 1.3229964682786333, 'is_coverage': 0.7621776504297995, 'oos_rmse': 1.3229964682786333, 'oos_coverage': 0.7621776504297995}


 11%|█         | 3/27 [01:43<13:53, 34.74s/it]

...

{'neighborhood_size': 8192, 'is_rsquared': 0.0271513606959376, 'is_rmse': 1.2988674356911376, 'is_coverage': 0.997134670487106, 'oos_rmse': 1.2988674356911376, 'oos_coverage': 0.997134670487106}


 96%|█████████▋| 26/27 [22:30<01:34, 94.74s/it]

{'neighborhood_size': 11585, 'is_rsquared': 0.027154311568203693, 'is_rmse': 1.2991640668554478, 'is_coverage': 0.997134670487106, 'oos_rmse': 1.2991640668554478, 'oos_coverage': 0.997134670487106}


100%|██████████| 27/27 [25:09<00:00, 55.92s/it] 

{'neighborhood_size': 13865, 'is_rsquared': 0.027171678718070247, 'is_rmse': 1.299150159384227, 'is_coverage': 0.997134670487106, 'oos_rmse': 1.299150159384227, 'oos_coverage': 0.997134670487106}


In [ ]:
allerrors = pd.concat(errors_by_neighborhood_size, ignore_index=True)

## Visualizations

In [ ]:
wide_data = pd.melt(allerrors, ["neighborhood_size", "signal"])

In [ ]:
for signal in wide_data["signal"].unique():
    wide_data.loc[lambda x: x["signal"] == signal, "variable"] = (
        f"{signal}_" + wide_data.loc[lambda x: x["signal"] == signal, "variable"]
    )

In [ ]:
plt.figure(figsize=(20, 10))
_ = sns.lineplot(
    x="neighborhood_size",
    y="value",
    hue="variable",
    data=wide_data.loc[lambda x: x.variable.str.contains("coverage")],
).set(xscale="log", title="Prediction Coverage")

In [ ]:
plt.figure(figsize=(20, 10))
_ = sns.lineplot(
    x="neighborhood_size",
    y="value",
    hue="variable",
    data=wide_data.loc[lambda x: x.variable.str.contains("rmse")],
).set(xscale="log", title="Root Mean Squared Error")

In [ ]:
allerrors.groupby(["signal", "neighborhood_size"]).mean().sort_values(
    by="oos_rmse"
).head(50)

In [ ]:
allerrors.groupby(["signal", "neighborhood_size"]).mean().rolling(3).mean().xs('item', level='signal').sort_values(by='oos_rmse')

In [ ]:
(1.199971 + 1.200345 + 1.200527) / 3

In [ ]:
(1.198847 + 1.198858 + 1.202293 ) / 3

## Store best hyperparameters

In [ ]:
allerrors.groupby(["signal", "neighborhood_size"]).mean().sort_values(
    by="oos_rmse"
).reset_index().groupby("signal").first()

In [ ]:
outdir = "parameters"
if not os.path.exists(outdir):
    os.mkdir(outdir)
os.chdir(outdir)

In [ ]:
allerrors.groupby(["signal", "neighborhood_size"]).mean().sort_values(
    by="oos_rmse"
).reset_index().groupby("signal").first().to_pickle("neighborhoodcf.best.pkl")
allerrors.to_pickle("neighborhoodcf.all.pkl")